## Initiate the vissim instance

In [1]:
# COM-Server
import win32com.client as com
import igraph
import qgrid
from VISSIM_helpers import VissimRoadNet
from os.path import abspath, join, exists
import os
from shutil import copyfile
import pandas as pd
import math

Add autocompletion for VISSIM COM Object

In [2]:
from IPython.utils.generics import complete_object

@complete_object.register(com.DispatchBaseClass)
def complete_dispatch_base_class(obj, prev_completions):
    try:
        ole_props = set(obj._prop_map_get_).union(set(obj._prop_map_put_))
        return list(ole_props) + prev_completions
    except AttributeError:
        pass

Start Vissim and load constants

In [3]:
Vissim = com.gencache.EnsureDispatch("Vissim.Vissim")
from win32com.client import constants as c

Setting the parameters used for simulation

In [4]:
DTA_Parameters = dict(
    # DTA Parameters
    EvalInt = 600, # seconds
    ScaleTotVol = False,
    ScaleTotVolPerc = 1,
    CostFile = 'costs.bew',
    ChkEdgOnReadingCostFile = True,
    PathFile = 'paths.weg',
    ChkEdgOnReadingPathFile = True,
    CreateArchiveFiles = True,
    VehClasses = '',
)

# Simulation parameters
Sim_Parameters = dict(
    NumRuns = 1,
    RandSeedIncr = 0,
    UseMaxSimSpeed = True,
    SimBreakAt = 600,
    NumCores = 8,
)

FileName = abspath(r"..\SO sim files\Vol100per.inpx")
WorkingFolder = abspath(r"..\SO sim files")

Resetting edge and path cost files

In [5]:
default_cost_file = abspath('..\SO sim files\costs_020.bew')
defualt_path_file = abspath('..\SO sim files\paths_020.weg')

current_cost_file = abspath(join(WorkingFolder, DTA_Parameters['CostFile']))
if exists(current_cost_file):
    os.remove(current_cost_file)
copyfile(default_cost_file, current_cost_file)

current_path_file = abspath(join(WorkingFolder, DTA_Parameters['PathFile']))
if exists(current_path_file):
    os.remove(current_path_file)
copyfile(defualt_path_file, current_path_file)

'C:\\Users\\ITSLab\\Documents\\Oliver Liang\\SO sim files\\paths.weg'

Load the test network

In [6]:
Vissim.LoadNet(FileName)

Read dynamic assignment network

In [7]:
vis_net = Vissim.Net
vis_net.Paths.ReadDynAssignPathFile()

network_graph = VissimRoadNet(vis_net)

Check if dynamic assignment graph has changed

In [8]:
ref_edge_list = pd.read_pickle("edges_attr.pkl.gz")
assert (network_graph.visedges['ToNode'] == ref_edge_list['ToNode']).all()

We start by opening the network to be tested and adjust its settings

In [9]:
DynamicAssignment = Vissim.Net.DynamicAssignment
for attname, attvalue in DTA_Parameters.items():
    DynamicAssignment.SetAttValue(attname, attvalue)
    
Simulation = Vissim.Net.Simulation
for attname, attvalue in Sim_Parameters.items():
    Simulation.SetAttValue(attname, attvalue)

Run first DTA period as usual

In [10]:
Vissim.Graphics.CurrentNetworkWindow.SetAttValue("QuickMode", 1)
Simulation.RunContinuous()

In [11]:
def current_period():
    return int(math.ceil(Vissim.Simulation.SimulationSecond / Vissim.Net.DynamicAssignment.AttValue('EvalInt')))

Run simulation with custom route assignment

In [ ]:
paths = {}
Vissim.Simulation.RunSingleStep()
while True:
    new_vehs = vis_net.Vehicles.GetDeparted()
    network_graph.es['weight'] = [edge[1] if edge[1] else 99999. for edge in vis_net.Edges.GetMultiAttValues(f'TravTmRaw({current_period() - 1})')]
    for veh in new_vehs:
        origin_lot = int(veh.AttValue('OrigParkLot'))
        destination_lot = int(veh.AttValue('DestParkLot'))
        node_path = network_graph.parking_lot_route(origin_lot, destination_lot)
        vis_path = vis_net.Paths.AddPath(origin_lot, destination_lot, [str(node) for node in node_path[1:-1]])
        paths[veh.AttValue('No')] = node_path
    if Vissim.Simulation.SimulationSecond > 4499:
        break
    Vissim.Simulation.RunSingleStep()

Vissim.Simulation.RunContinuous()

In [14]:
Vissim.Exit()